In [1]:
import boto3
import xesmf as xe
import xarray as xr
import netCDF4
import pandas as pd
import argparse
import base64
import boto3
import json
import requests
import os
import platform
from subprocess import Popen
import shutil
from datetime import datetime
import re
import pandas as pd
from load_credentials import *
from bbox import * 
from netCDF4 import Dataset
import netCDF4 as nc
from bbox import * 
import numpy as np


# Define event with credentials and S3 endpoint details
event = {
    's3_endpoint': 'https://data.asdc.earthdata.nasa.gov/s3credentials',  # replace with actual endpoint
    'edl_username': 'amanda.murray19',  # replace with your EDL username
    'edl_password': 'Sat_modeling_berk2024',  # replace with your EDL password
    'bucket_name': 'asdc-prod-protected/TEMPO/TEMPO_NO2_L3_V03'  # replace with your bucket name
}

netrc_path = os.path.expanduser('~/.netrc')  # Expands to the user's home directory
if os.path.exists(netrc_path):
    print(".netrc file exists.")
    os.remove(netrc_path)
    print(".netrc file has been removed.")
else:
    print(".netrc file does not exist.")# Retrieve credentials

.netrc file does not exist.


In [2]:
df = pd.read_csv("../../data/tempo_data/no2_tempo_files_df.csv")
df['time_hr_ct'] = pd.to_datetime(df['timestamp_ct']).dt.floor('h')

result_df = df.groupby(['date_central', 'Hour'], as_index=False).first()
result_df['time_hr_ct'] = pd.to_datetime(result_df['timestamp_ct']).dt.floor('h')
result_df[['time_hr_ct']].to_csv('../../data/tempo_data/no2_file_hours.csv')

In [ ]:
### TIME 5 days = 875 s (~15 min)
# Initialize the Google Cloud Storage client
# Example DataFrame (assuming it exists)
satellite_null_data = pd.DataFrame({
    'date': [],
    'file': [],
    'time': [],
    'null_count': [],
    'percent_null': []
})

date_list = list(df['date_central'].unique())

for date in date_list[25:54]:
    # Retrieve credentials
    creds = retrieve_credentials(event)

    # Use the credentials to access the S3 bucket
    client = boto3.client('s3',
        aws_access_key_id=creds["accessKeyId"],
        aws_secret_access_key=creds["secretAccessKey"],
        aws_session_token=creds["sessionToken"]
    )
    
    file_df = df[df['date_central']==date]
    filepaths = list(file_df['FilePath'])
    times = list(file_df['time_hr_ct'])
    full_time = list(file_df['timestamp_ct'])
    
    daily_xr_list = []
    
    for i in range(len(filepaths)):
        file=filepaths[i]
        new_time = str(np.array(times[i]))
        
        bucket_name = 'asdc-prod-protected'
        object_key = file
        local_file_name = 'current_sat.nc'
        client.download_file(bucket_name, object_key, local_file_name)
        
        ds = nc.Dataset('current_sat.nc', mode='r')
        support_data_xr = ds.groups['support_data']
        support_data_xr = xr.open_dataset(xr.backends.NetCDF4DataStore(support_data_xr))
        support_data_xr=support_data_xr[['vertical_column_total','eff_cloud_fraction']]
        tempo = xr.open_dataset('current_sat.nc', group="/product")
        tempo = xr.merge([tempo, support_data_xr])
        del ds
        del support_data_xr
        lat_lon_coords = xr.open_dataset('current_sat.nc')
        lat = lat_lon_coords.coords['latitude'].values
        lon = lat_lon_coords.coords['longitude'].values
        tempo = tempo.assign_coords(latitude=("latitude", lat), longitude=("longitude", lon), time=[new_time])
        del lat
        del lon

        # Filter the data using the .sel method for selecting ranges in xarray
        louisiana_data = tempo.sel(latitude=(tempo.latitude  >= lat_min) & (tempo.latitude <= lat_max))
        louisiana_data = louisiana_data.sel(longitude=(tempo.longitude  >= lon_min) & (tempo.longitude <= lon_max))
        del tempo
        
        mask = louisiana_data['main_data_quality_flag'] >1 
        louisiana_data = louisiana_data.where(~mask, other=np.nan)
        mask = louisiana_data['main_data_quality_flag'].isnull()
        louisiana_data = louisiana_data.where(~mask, other=np.nan)
        
        lat_new = np.arange(lat_min+0.005, lat_max, 0.01)
        lon_new = np.arange(lon_min+0.005, lon_max, 0.01)

        ds_out = xr.Dataset(
            {
                "lat": (["lat"], lat_new),
                "lon": (["lon"], lon_new),
            }
        )

        # Create the regridder object.
        regridder = xe.Regridder(louisiana_data, ds_out, "bilinear")

        # Apply the regridding operation.
        louisiana_data_regrid = regridder(louisiana_data)
        daily_xr_list.append(louisiana_data_regrid)
        
        null_count = int(louisiana_data_regrid.vertical_column_troposphere.isnull().sum().values)
        total_elements = louisiana_data_regrid.vertical_column_troposphere.size
        nan_percentage = round((null_count / total_elements) * 100,0)
        
        new_row = {'date':date, 'file':file, 'time':new_time, 'real_time':full_time, 'null_count':null_count, 'percent_null':nan_percentage}
        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])
        # Append the new row to the DataFrame using concat
        satellite_null_data = pd.concat([satellite_null_data, new_row_df], ignore_index=True)
        
    combined_data = xr.concat(daily_xr_list, dim='time')
    
    combined_data.to_netcdf(f'../../data/tempo_data/daily_files/tempo_{date}.nc')
         

satellite_null_data.to_csv("../../data/tempo_data/satellite_null_data_09_2023.csv")

<Response [307]>
<Response [307]>
<Response [307]>
<Response [307]>
<Response [307]>
<Response [307]>


In [ ]:
### TIME 5 days = 875 s (~15 min)
# Initialize the Google Cloud Storage client
# Example DataFrame (assuming it exists)
satellite_null_data = pd.DataFrame({
    'date': [],
    'file': [],
    'time': [],
    'null_count': [],
    'percent_null': []
})

date_list = list(df['date_central'].unique())

for date in date_list[54:85]:
    # Retrieve credentials
    creds = retrieve_credentials(event)

    # Use the credentials to access the S3 bucket
    client = boto3.client('s3',
        aws_access_key_id=creds["accessKeyId"],
        aws_secret_access_key=creds["secretAccessKey"],
        aws_session_token=creds["sessionToken"]
    )
    
    file_df = df[df['date_central']==date]
    filepaths = list(file_df['FilePath'])
    times = list(file_df['time_hr_ct'])
    full_time = list(file_df['timestamp_ct'])
    
    daily_xr_list = []
    
    for i in range(len(filepaths)):
        file=filepaths[i]
        new_time = str(np.array(times[i]))
        
        bucket_name = 'asdc-prod-protected'
        object_key = file
        local_file_name = 'current_sat.nc'
        client.download_file(bucket_name, object_key, local_file_name)
        
        ds = nc.Dataset('current_sat.nc', mode='r')
        support_data_xr = ds.groups['support_data']
        support_data_xr = xr.open_dataset(xr.backends.NetCDF4DataStore(support_data_xr))
        support_data_xr=support_data_xr[['vertical_column_total','eff_cloud_fraction']]
        tempo = xr.open_dataset('current_sat.nc', group="/product")
        tempo = xr.merge([tempo, support_data_xr])
        del ds
        del support_data_xr
        lat_lon_coords = xr.open_dataset('current_sat.nc')
        lat = lat_lon_coords.coords['latitude'].values
        lon = lat_lon_coords.coords['longitude'].values
        tempo = tempo.assign_coords(latitude=("latitude", lat), longitude=("longitude", lon), time=[new_time])
        del lat
        del lon

        # Filter the data using the .sel method for selecting ranges in xarray
        louisiana_data = tempo.sel(latitude=(tempo.latitude  >= lat_min) & (tempo.latitude <= lat_max))
        louisiana_data = louisiana_data.sel(longitude=(tempo.longitude  >= lon_min) & (tempo.longitude <= lon_max))
        del tempo
        
        mask = louisiana_data['main_data_quality_flag'] >1 
        louisiana_data = louisiana_data.where(~mask, other=np.nan)
        mask = louisiana_data['main_data_quality_flag'].isnull()
        louisiana_data = louisiana_data.where(~mask, other=np.nan)
        
        lat_new = np.arange(lat_min+0.005, lat_max, 0.01)
        lon_new = np.arange(lon_min+0.005, lon_max, 0.01)

        ds_out = xr.Dataset(
            {
                "lat": (["lat"], lat_new),
                "lon": (["lon"], lon_new),
            }
        )

        # Create the regridder object.
        regridder = xe.Regridder(louisiana_data, ds_out, "bilinear")

        # Apply the regridding operation.
        louisiana_data_regrid = regridder(louisiana_data)
        daily_xr_list.append(louisiana_data_regrid)
        
        null_count = int(louisiana_data_regrid.vertical_column_troposphere.isnull().sum().values)
        total_elements = louisiana_data_regrid.vertical_column_troposphere.size
        nan_percentage = round((null_count / total_elements) * 100,0)
        
        new_row = {'date':date, 'file':file, 'time':new_time, 'real_time':full_time, 'null_count':null_count, 'percent_null':nan_percentage}
        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])
        # Append the new row to the DataFrame using concat
        satellite_null_data = pd.concat([satellite_null_data, new_row_df], ignore_index=True)
        
    combined_data = xr.concat(daily_xr_list, dim='time')
    
    combined_data.to_netcdf(f'../../data/tempo_data/daily_files/tempo_{date}.nc')
         

satellite_null_data.to_csv("../../data/tempo_data/satellite_null_data_10_2023.csv")

In [ ]:
### TIME 5 days = 875 s (~15 min)
# Initialize the Google Cloud Storage client
# Example DataFrame (assuming it exists)
satellite_null_data = pd.DataFrame({
    'date': [],
    'file': [],
    'time': [],
    'null_count': [],
    'percent_null': []
})

date_list = list(df['date_central'].unique())

for date in date_list[259:290]:
    # Retrieve credentials
    creds = retrieve_credentials(event)

    # Use the credentials to access the S3 bucket
    client = boto3.client('s3',
        aws_access_key_id=creds["accessKeyId"],
        aws_secret_access_key=creds["secretAccessKey"],
        aws_session_token=creds["sessionToken"]
    )
    
    file_df = df[df['date_central']==date]
    filepaths = list(file_df['FilePath'])
    times = list(file_df['time_hr_ct'])
    full_time = list(file_df['timestamp_ct'])
    
    daily_xr_list = []
    
    for i in range(len(filepaths)):
        file=filepaths[i]
        new_time = str(np.array(times[i]))
        
        bucket_name = 'asdc-prod-protected'
        object_key = file
        local_file_name = 'current_sat.nc'
        client.download_file(bucket_name, object_key, local_file_name)
        
        ds = nc.Dataset('current_sat.nc', mode='r')
        support_data_xr = ds.groups['support_data']
        support_data_xr = xr.open_dataset(xr.backends.NetCDF4DataStore(support_data_xr))
        support_data_xr=support_data_xr[['vertical_column_total','eff_cloud_fraction']]
        tempo = xr.open_dataset('current_sat.nc', group="/product")
        tempo = xr.merge([tempo, support_data_xr])
        del ds
        del support_data_xr
        lat_lon_coords = xr.open_dataset('current_sat.nc')
        lat = lat_lon_coords.coords['latitude'].values
        lon = lat_lon_coords.coords['longitude'].values
        tempo = tempo.assign_coords(latitude=("latitude", lat), longitude=("longitude", lon), time=[new_time])
        del lat
        del lon

        # Filter the data using the .sel method for selecting ranges in xarray
        louisiana_data = tempo.sel(latitude=(tempo.latitude  >= lat_min) & (tempo.latitude <= lat_max))
        louisiana_data = louisiana_data.sel(longitude=(tempo.longitude  >= lon_min) & (tempo.longitude <= lon_max))
        del tempo
        
        mask = louisiana_data['main_data_quality_flag'] >1 
        louisiana_data = louisiana_data.where(~mask, other=np.nan)
        mask = louisiana_data['main_data_quality_flag'].isnull()
        louisiana_data = louisiana_data.where(~mask, other=np.nan)
        
        lat_new = np.arange(lat_min+0.005, lat_max, 0.01)
        lon_new = np.arange(lon_min+0.005, lon_max, 0.01)

        ds_out = xr.Dataset(
            {
                "lat": (["lat"], lat_new),
                "lon": (["lon"], lon_new),
            }
        )

        # Create the regridder object.
        regridder = xe.Regridder(louisiana_data, ds_out, "bilinear")

        # Apply the regridding operation.
        louisiana_data_regrid = regridder(louisiana_data)
        daily_xr_list.append(louisiana_data_regrid)
        
        null_count = int(louisiana_data_regrid.vertical_column_troposphere.isnull().sum().values)
        total_elements = louisiana_data_regrid.vertical_column_troposphere.size
        nan_percentage = round((null_count / total_elements) * 100,0)
        
        new_row = {'date':date, 'file':file, 'time':new_time, 'real_time':full_time, 'null_count':null_count, 'percent_null':nan_percentage}
        # Convert the new row to a DataFrame
        new_row_df = pd.DataFrame([new_row])
        # Append the new row to the DataFrame using concat
        satellite_null_data = pd.concat([satellite_null_data, new_row_df], ignore_index=True)
        
    combined_data = xr.concat(daily_xr_list, dim='time')
    
    combined_data.to_netcdf(f'../../data/tempo_data/daily_files/tempo_{date}.nc')
         

satellite_null_data.to_csv("../../data/tempo_data/satellite_null_data_05_2024.csv")

<Response [307]>
<Response [307]>
<Response [307]>


In [ ]:
# import matplotlib.pyplot as plt
# # Open the file with xarray
# ds = xr.open_dataset('daily.nc')
# # Assuming your xarray is called 'x'
# # Define the bounding box for Louisiana
# lat_min, lat_max = 28.92, 33.02
# lon_min, lon_max = -94.04, -88.82

# subset = ds.sel(time='2023-08-07 07:00:00')

# # Plot the weight variable in Louisiana's bounding box
# plt.figure(figsize=(8, 6))
# subset.vertical_column_troposphere.plot(cmap='viridis')
# plt.title('Weight Variable in Louisiana Bounding Box')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.grid(True)
# plt.show()